In [43]:
# Importing required libraries
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
import smtplib
import time
import datetime
List_DB=[]

def check_float(potential_float):
    try:
        float(potential_float)
        return True
    except ValueError:
        return False

def get_url_html_data(URL_to_Connect,heads,List_DB):
    page=requests.get(URL_to_Connect,headers=heads)
    Soup1=BeautifulSoup(page.content,"html.parser")
    Soup1=BeautifulSoup(Soup1.prettify(),"html.parser")
    url_2=Soup1.find_all("a",href=True,class_="a-link-normal a-text-normal")
    for links in (url_2):
        url_new=str(str("https://www.amazon.in")+str(links['href']))
        page=requests.get(url_new,headers=heads)
        Soup1=BeautifulSoup(page.content,"html.parser")
        Soup1=BeautifulSoup(Soup1.prettify(),"html.parser")
        data1=Soup1.find("span",id="productTitle")
        if (data1!=None):
            data1=Soup1.find("span",id="productTitle").get_text().strip()
            data_1=data1.split(" ")
            BrandName=data_1.pop(0)            
            if (len(data_1)>=2):
                if (check_float(data_1[0])):
                    WidthName=float(data_1[0])
                elif(check_float(data_1[1])):
                    WidthName=float(data_1[1])
                else:
                    WidthName=" "
                if (")" in data1):
                    Comma_Ind=data1.index(")")                
                    DescName=data1[Comma_Ind+1:]
                    data4=Soup1.find("span",id="priceblock_ourprice")
                    if (data4!=None):
                        PriceName=data4.get_text().strip()
                    else:
                        data4=Soup1.find("span",id="priceblock_dealprice")
                        if (data4!=None):
                            PriceName=data4.get_text().strip()
                        else:
                            continue
                    data5=Soup1.find("span",class_="a-icon-alt")
                    if (data5!=None):
                        data5=data5.get_text().strip().split(" ")[0]
                        StarName=data5
                    else:
                        StarName=" "
                    List_DB.append([BrandName,WidthName,PriceName,StarName,DescName])
        else:
            continue
#     print(List_DB)
    return Soup1

def go_to_next_page(Soup):
    page=requests.get(Soup,headers=heads)
    Soup1=BeautifulSoup(page.content,"html.parser")
    Soup1=BeautifulSoup(Soup1.prettify(),"html.parser")
    page=Soup1.find("ul",class_="a-pagination")
    if not page.find("li",{"class":"a-disabled a-last"}):
        url="https://www.amazon.in/"+str(page.find("li", {"class":"a-last"}).find("a")["href"])
        return url
    else:
        return

URL_to_Connect="https://www.amazon.in/s?k=televisions&i=electronics&rh=n%3A976419031%2Cp_89%3ALG%7CMI%7COnePlus%7COnida%7CSamsung%7CSony%7CTCL%7CTOSHIBA&dc&qid=1637476006&rnid=3837712031&ref=sr_nr_p_89_6"
heads={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "Accept-Encoding": "gzip, deflate, br","DNT":"1","Connection":"close","Upgrade-Insecure-Requests": "1"}

# HTML_Current=get_url_html_data(URL_to_Connect,heads,List_DB)
Kount=0
while True:
    HTML_Current=get_url_html_data(URL_to_Connect,heads,List_DB)
    Link_Next=go_to_next_page(URL_to_Connect)
    print("Page:",Kount)
    Kount+=1
    if Link_Next==None:
        break
    URL_to_Connect=Link_Next

Page: 0
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6


In [44]:
len(List_DB)

146

In [45]:
cols_names=["Brand","Width (cms)","Price (INR)","Star Ratings (Out of 5)", "Description"]
Df=pd.DataFrame(columns=cols_names)

for i in range(len(List_DB)):
    Df.loc[i]=List_DB[i]

In [46]:
Df

,Brand,Width (cms),Price (INR),Star Ratings (Out of 5),Description
0,LG,108,"₹37,499.00",4.3,4K Ultra HD Smart LED TV 43UP7500PTZ (Rocky B...
1,Mi,100,"₹24,999.00",4.3,Horizon Edition Full HD Android LED TV 4A | L...
2,OnePlus,80,"₹18,999.00",4.2,Y Series HD Ready LED Smart Android TV 32Y1 (...
3,Samsung,80,"₹18,490.00",4.3,Wondertainment Series HD Ready LED Smart TV U...
4,Mi,80,"₹16,999.00",4.3,HD Ready Android Smart LED TV 4A PRO | L32M5-...
...,...,...,...,...,...
141,LG,,"₹19,999.00",3.9,", Full HD (1920 x 1080) TV Monitor, Inbuilt Sp..."
142,LG,139.7,"₹75,990.00",4.2,Smart Ultra HD 4K LED TV 55UN7190PTA (2020 Mo...
143,LG,139,"₹1,24,999.00",3.5,4K Ultra HD Smart LED TV 55SK8500PTA (Black)
144,LG,190.5,"₹3,29,270.00",3.0,75SK8000PTA 4K Super UHD LED Smart TV (Black)


In [47]:
Df.to_excel("TVs_DB Amazon.xlsx")